# Data import 

Where I develop the functions to import the data - from .wav files, into tensors.

In [1]:
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader, TensorDataset
import torchaudio

load_dotenv()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/jimbo/Documents/coding/misc/audio-deepfake-detection/venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/jimbo/Documents/coding/misc/audio-deepfake-detection/venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/jimbo/Documents/coding/misc/audio-deepfake-detect

True

In [2]:
data_dir = os.getenv("DATA_DIR")
temp_file = '1.wav'

In [3]:
transform = torchaudio.transforms.MelSpectrogram(16000)

/Users/jimbo/Documents/coding/misc/audio-deepfake-detection/venv/lib/python3.11/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [4]:
def pad_tensor(tensor, target_length):
    '''
    Given a tensor and a target length, pads the tensor so that its length a multiple of the target length -
    so that it can be split into equal parts.

    Parameters
    ----------
    tensor : torch.tensor
        The tensor to pad
    target_length : int
        The target length to pad the tensor to

    Returns
    -------
    torch.tensor
        The padded tensor
    '''
    
    _, length = tensor.shape
    if length % target_length != 0:
        # Calculate padding needed
        padding_needed = target_length - (length % target_length)
        # Pad the tensor
        tensor = torch.nn.functional.pad(tensor, (0, padding_needed))
    return tensor

In [5]:
def load_file(file, target_length=80):
    '''
    Given a file path, loads the file, creates a mel spectrogram, pads it to be cleanly divisible by target_length,
    and splits it into chunks of target_length.

    Parameters
    ----------
    file : str
        The file path to load
    target_length : int
        The target length to split the tensor into

    Returns
    -------
    tuple of torch.tensor
        A tuple of tensors, each of length target_length
    '''

    # load wav file
    waveform, _ = torchaudio.load(file, normalize=True)
    waveform = waveform.squeeze(0)

    # create mel spectrogram
    mel_specgram = transform(waveform)

    # pad tensor so it's cleanly divisible by target_length
    padded_tensor = pad_tensor(mel_specgram, target_length)

    # return the tensor, split into target_length chunks
    return padded_tensor.split(target_length, dim=1)

In [6]:
# load wav file
waveform, sample_rate = torchaudio.load(os.path.join(data_dir, temp_file), normalize=True)
waveform = waveform.squeeze()

In [7]:
mel_specgram = transform(waveform)

In [8]:
mel_specgram.shape

torch.Size([128, 888])

In [9]:
new_tens = pad_tensor(mel_specgram, 80)

In [10]:
len(new_tens.split(80, dim=1))

12

In [11]:
tensors_loaded = load_file(os.path.join(data_dir, temp_file))

In [27]:
def load_dataset(meta_file, 
                 target_length=80, 
                 batch_size=32,
                 files_to_load='all'):
    '''
    Given a meta file, loads in the dataset.

    Parameters
    ----------
    meta_file : str
        The meta file to load
    target_length : int
        The target length to split the tensor into
    batch_size : int
        The batch size to use for the DataLoader
    files_to_load : int or 'all'
        The number of files to load. If 'all', all files are loaded.

    Returns
    -------
    tuple of torch.tensor
        A tuple of tensors - the positives and negatives
    '''

    positives = []
    negatives = []

    # Load the meta file
    meta = pd.read_csv(meta_file)

    # iterate through the files
    for i, row in enumerate(meta.itertuples()):
        
        # If we're only loading a subset of the files, check if we've loaded enough
        if files_to_load != 'all' and i >= files_to_load:
            break

        # Load the file
        file_tensors = load_file(os.path.join(data_dir, row.file), target_length=target_length)

        # Add the tensors to the appropriate list
        for tensor in file_tensors:
            if row.numeric_label == 1:
                positives.append(tensor)
            else:
                negatives.append(tensor)

    # Create labels
    positive_labels = torch.tensor([[1.0, 0.0]] * len(positives))
    negative_labels = torch.tensor([[0.0, 1.0]] * len(negatives))

    # Combine the inputs
    data = torch.cat((torch.stack(positives), torch.stack(negatives)), dim=0)
    # Combine the labels
    labels = torch.cat((positive_labels, negative_labels), dim=0)

    # Create a dataset
    dataset = TensorDataset(data, labels)

    # Create a dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # return the dataloader
    return dataloader

In [28]:
dataloader = load_dataset('train_meta.csv', files_to_load=100)

In [31]:
for batch_data, batch_labels in dataloader:
    print(f"Batch data shape: {batch_data.shape}")
    print(f"Batch labels: {batch_labels}")

Batch data shape: torch.Size([32, 128, 80])
Batch labels: tensor([[1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.]])
Batch data shape: torch.Size([32, 128, 80])
Batch labels: tensor([[1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
 